In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut

In [30]:
cancer_data = pd.read_csv("cancer.csv")
print(cancer_data.shape)
cancer_labels = cancer_data['label'].values
cancer_features = cancer_data.drop(['label'], axis=1).values
cancer_data.head()

(569, 31)


,label,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [55]:
spam_data = pd.read_csv("spam.csv")
print(spam_data.shape)
spam_labels = spam_data['label'].values
spam_features = spam_data.drop(['label'], axis=1).values
spam_data.head()

(4601, 58)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_:,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,label
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [61]:
def dist(x1, x2):
    return np.linalg.norm(x1 - x2)


class KNN:
    def __init__(self, k=3):
        self.k = k
        
    def fit(self, X, y):
        self.data = X
        self.ys = y
        
    def predict(self, xs):
        return np.array([self.get_label(self.get_neighbours(x)) for x in xs])
    
    def set_k(self, k):
        self.k = k
    
    def get_neighbours(self, x):
        return sorted(enumerate(self.data), key=lambda y: dist(x, y[1]))[:self.k]
    
    def get_label(self, x):
        raise NotImplementedError()
        

class KNNClassifier(KNN):
    def __init__(self, k=3):
        super().__init__(k)
        
    def get_label(self, neighbours):
        labels = {}
        for i, x in neighbours:
            if self.ys[i] not in labels:
                labels[self.ys[i]] = 0
            labels[self.ys[i]] += 1
        return sorted(list(labels.items()), key=lambda t: t[1])[0][0]

In [64]:
def accuracy(y_pr, y_te):
    return np.mean(y_pr == y_te)

def evaluate_knn(X, y, ks, train_ratio=0.75):
    loo = LeaveOneOut()
    match = np.zeros(len(ks))
    max_k = max(ks)
    for tr_ind, te_ind in loo.split(X):
        X_tr, X_te = X[tr_ind], X[te_ind]
        y_tr, y_te = y[tr_ind], y[te_ind]
        knn = KNNClassifier(max_k)
        knn.fit(X_tr, y_tr)
        neighbours = knn.get_neighbours(X_te[0])
        for i, k in enumerate(ks):
            match[i] += 1 if knn.get_label(neighbours[:k]) == y_te[0] else 0
    return match / loo.get_n_splits(X)

In [65]:
cancer_accuracy_1_10 = evaluate_knn(cancer_features, cancer_labels, list(range(1, 11)))
print(cancer_accuracy_1_10)

[0.91564148 0.91564148 0.89982425 0.87170475 0.84885764 0.82776801
 0.80843585 0.7943761  0.79086116 0.77680141]


In [67]:
spam_accuracy_1_10 = evaluate_knn(spam_features, spam_labels, list(range(1, 11)))
print(spam_accuracy_1_10)

[0.83047164 0.83047164 0.73288416 0.70093458 0.62964573 0.61769181
 0.56596392 0.55314062 0.51119322 0.50836775]
